## Modelling the sea using grid-based methods

In [1]:
from pygradu import gridify
import shapely.geometry
import pandas as pd
import numpy as np
import importlib

In [2]:
grid = gridify.area_to_grid(side_length=5000)

Original NE: POINT (134 40)
Original SW: POINT (123 32)
Transformed NE: (798015.8733194428, 745576.647904759)
Transformed SW: (-178108.15701596113, -158644.45983528974)
col_count 195
row_count 180
grid_size 35100


/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [3]:
grid.extract_coords_lat_lon(2)

[53.104092447512414, 8.51290159496569]